In [1]:
import os
import json
import pandas as pd
import geopandas as gpd
from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon

import json
import matplotlib as mpl
import pylab as plt
import string
import numpy as np

from bokeh.io import output_file, show, output_notebook, export_png
from bokeh.layouts import row, column, layout
from bokeh.models import CustomJS, GeoJSONDataSource, LogColorMapper, ColorBar, HoverTool
from bokeh.plotting import figure, save, output_file
from bokeh.palettes import brewer
from bokeh.tile_providers import CARTODBPOSITRON, CARTODBPOSITRON_RETINA, STAMEN_TONER, get_provider
from bokeh.models.widgets import Select

import plotly.express as px

## Read in the data

In [2]:
# Define the filepaths
home_dir = os.path.expanduser("~")
data_fp = "iec/shrug_covid"

# Read in the COVID data
df = pd.read_stata(os.path.join(home_dir, data_fp, "hospitals_dist_export.dta"))

# Read in the pc11 district shapefile
gdf = gpd.read_file(os.path.join(home_dir, "iec1", "gis", "pc11", "pc11-district.shp"))

# Rename columns to match pc11 keys
gdf = gdf.rename(columns = {"pc11_s_id": "pc11_state_id",
                            "pc11_d_id": "pc11_district_id"})


data = gdf.merge(df, left_on=["pc11_state_id", "pc11_district_id"], right_on=["pc11_state_id", "pc11_district_id"])

# convert multipoloygon states into individual polygons
data = data.explode()

# reduce resolution of polygons to make loading faster
data['geometry'] = data['geometry'].simplify(tolerance=0.01)

data['pc11_state_name'] = [string.capwords(x) for x in data['pc11_state_name']]
data['pc11_district_name'] = [string.capwords(x) for x in data['pc11_district_name']]

# convert to web mercator crs to match the basemap
data = data.to_crs(epsg=3857)

# write file to geojson
# data.to_file("/scratch/acampion/test.json", driver="GeoJSON")

## Plot the Data

In [6]:
def get_geodatasource(gdf,
                      index_columns=['pc11_state_name', 'pc11_state_id', 'pc11_district_name','pc11_district_id']):    
    """Get getjsondatasource from geopandas object""" 
    json_data = json.dumps(json.loads(gdf.to_json()))
    return GeoJSONDataSource(geojson = json_data)


def plot_patch_data(geosource, df,
                    column="", column_label="data",
                    cmap="YlGnBu", cmap_res=8, cmap_r=True, 
                    title='', ph=850, pw=850):
        
    #-----# 
    # MAP #
    #-----# 
    # define tools
    tools = 'wheel_zoom, pan, reset'
    
    # define basemap
    tile_provider = get_provider(CARTODBPOSITRON_RETINA)

    # create figure
    p = figure(title = title,
               plot_height=ph ,
               plot_width=pw,
               x_axis_type="mercator",
               y_axis_type="mercator",
               toolbar_location='right',
               tools=tools)
    p.add_tile(tile_provider)
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None

    #-----------# 
    # COLOR BAR #
    #-----------# 
    # define the color palette
    palette = brewer[cmap][cmap_res]
    
    # reverese the color palette if desired
    if cmap_r == True:
        palette = palette[::-1]
    
    # extract data values for desired column
    vals = df[column]
    
    #Instantiate LogColorMapper that maps numbers in a range, into a sequence of colors.
    color_mapper = LogColorMapper(palette = palette, low = vals.min(), high = vals.max())
    
    # set the nan_color of the color mapper
    color_mapper.nan_color = "#f0f0f0"
    
    color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8, width=int(pw*0.9), height=20,
                         location=(0,0), orientation='horizontal', title="Number of Hospital Beds per 1000 People (log scale)")
    
    #Add patch renderer to figure
    p.patches('xs','ys',
              source=geosource,
              fill_alpha=0.7,
              line_width=0.5,
              line_color='black', 
              hover_fill_alpha=1.0,
              hover_line_width=2.0,
              hover_line_color='white',
              fill_color={'field': column, 'transform': color_mapper})
    
    #create hover tool
    hover_tool = HoverTool(
        tooltips=[
            ("State", "@pc11_state_name"),
            ("District", "@pc11_district_name"),
            (column_label, "@"+ column +"{1.11}")
        ])
    p.tools.append(hover_tool)
        
    #Specify figure layout.
    p.add_layout(color_bar, 'below')
    
    return p


In [8]:
output_file("bokeh-test.html", title="Number of Hospital Beds per 1000")
geosource = get_geodatasource(data)
p = plot_patch_data(geosource, df,
                     column="dlhs4_perk_beds_pubpriv", column_label="Hospital Beds per 1000 People",
                     title="Hospital Beds")
#save(p)
export_png(p, filename="../assets/dlhs4_perk_beds_pubpriv.png", height=1200, width=1200)

'/dartfs-hpc/rc/home/y/f00473y/ddl/covid/assets/dlhs4_perk_beds_pubpriv.png'

# Sandbox

Everything below here is under development and being used for testing but is not final code

### Get better color scaling

In [ ]:
column="dlhs4_perk_beds_pubpriv"
qcuts = pd.qcut(list(data[column].dropna()), 8)
data[f"{column}_cat"] = np.nan
data.loc[data[column].notnull(), f"{column}_cat"] = qcuts._codes

### JS callbacks in Bokeh

In [424]:
select = Select(title="Variables:", value="dlhs4_perk_beds_pubpriv", options=["dlhs4_perk_beds_pubpriv"])

callback = CustomJS(args={'geosource':geosource},code="""
        // print the selectd value of the select widget - 
        // this is printed in the browser console.
        // cb_obj is the callback object, in this case the select 
        // widget. cb_obj.value is the selected value.
        console.log('changed selected variable', cb_obj.value);

        // create a new variable for the data of the column data source
        // this is linked to the plot
        var data = geosource.data;

        // allocate the selected column to the field for the y values
        data['val'] = data[cb_obj.value];

        // register the change - this is required to process the change in 
        // the y values
        geosource.change.emit();
""")

l = column(p, dropdown)

### Plotly

In [442]:
home_dir = os.path.expanduser("~")
data_fp = "iec/shrug_covid"

# Read in the COVID data
df = pd.read_stata(os.path.join(home_dir, data_fp, "hospitals_dist_export.dta"))

# Read in the pc11 district shapefile
gdf = gpd.read_file(os.path.join(home_dir, "iec1", "gis", "pc11", "pc11-district.shp"))

# Rename columns to match pc11 keys
gdf = gdf.rename(columns = {"pc11_s_id": "pc11_state_id",
                            "pc11_d_id": "pc11_district_id"})


data = gdf.merge(df, left_on=["pc11_state_id", "pc11_district_id"], right_on=["pc11_state_id", "pc11_district_id"])

# convert multipoloygon states into individual polygons
data = data.explode()

# reduce resolution of polygons to make loading faster
data['geometry'] = data['geometry'].simplify(tolerance=0.02)
data['pc11_state_name'] = [string.capwords(x) for x in data['pc11_state_name']]
data['pc11_district_name'] = [string.capwords(x) for x in data['pc11_district_name']]

data.to_file("/scratch/acampion/test.json", driver="GeoJSON")
with open("/scratch/acampion/test.json") as json_file:
    districts = json.load(json_file)

In [ ]:
fig = px.choropleth_mapbox(df, geojson=districts, locations='pc11_district_id', color='dlhs4_perk_beds_pubpriv',
                           color_continuous_scale="Viridis",
                           range_color=(0, 80),
                           mapbox_style="carto-positron", featureidkey="properties.pc11_district_id",
                           zoom=3, center = {"lat": 20.5937, "lon": 78.9629},
                           opacity=0.5,
                           labels={'dlhs4_perk_beds_pubpriv':'Number of Hospital Beds'},
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})